In [3]:
import os
import pandas as pd

# %load_ext rpy2.ipython - no R env, python only, R 3.6 - from base 
# conda bi, server = 1,cpu - nogit
# %config IPCompleter.greedy=True

%config Completer.use_jedi = False

## Импорт данных

In [46]:
%cd /home/gladkov/storage/trash/BI/BP/RNA/
%ll -hs

/home/gladkov/storage/trash/BI/BP/RNA
total 2.2G
4.0K -rw-rw-r-- 1 gladkov 1.4K Apr  2 03:05 deseq2.r
4.0K -rw-rw-r-- 1 gladkov  770 Apr  2 03:05 draw-heatmap.r
 12M -rw-rw-r-- 1 gladkov  12M Apr  9  2018 GCF_000146045.2_R64_genomic.fna
3.7M -rw-rw-r-- 1 gladkov 3.7M Apr  9  2018 GCF_000146045.2_R64_genomic.fna.gz
8.9M -rw-rw-r-- 1 gladkov 8.9M May 16  2020 GCF_000146045.2_R64_genomic.gff
1.6M -rw-rw-r-- 1 gladkov 1.6M May 16  2020 GCF_000146045.2_R64_genomic.gff.gz
4.0K -rw-rw-r-- 1 gladkov  394 Apr  2 06:18 genes.txt
4.0K -rw-rw-r-- 1 gladkov 3.7K Apr  2 06:19 mapper_genes_26968_slimTab.txt
476K -rw-rw-r-- 1 gladkov 476K Apr  2 06:01 norm-matrix-deseq2.txt
276K -rw-rw-r-- 1 gladkov 276K Apr  2 06:02 output.pdf
388K -rw-rw-r-- 1 gladkov 387K Apr  2 05:58 ref_ann_count.txt
4.0K -rw-rw-r-- 1 gladkov  563 Apr  2 05:58 ref_ann_count.txt.summary
2.3M -rw-rw-r-- 1 gladkov 2.3M Apr  2 05:45 ref.gtf
4.0K drwxrwxr-x 2 gladkov 4.0K Apr  2 05:22 ref_hisat_ind/
816K -rw-rw-r-- 1 gladkov 814K Apr 

In [9]:
%%bash

wget http://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/146/045/GCF_000146045.2_R64/GCF_000146045.2_R64_genomic.fna.gz
wget ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/146/045/GCF_000146045.2_R64/GCF_000146045.2_R64_genomic.gff.gz
wget http://ftp.sra.ebi.ac.uk/vol1/fastq/SRR941/SRR941818/SRR941818.fastq.gz


--2021-04-02 04:58:56--  http://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/146/045/GCF_000146045.2_R64/GCF_000146045.2_R64_genomic.fna.gz
Resolving ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)... 165.112.9.229, 130.14.250.13, 2607:f220:41f:250::229, ...
Connecting to ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)|165.112.9.229|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/146/045/GCF_000146045.2_R64/GCF_000146045.2_R64_genomic.fna.gz [following]
--2021-04-02 04:58:57--  https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/146/045/GCF_000146045.2_R64/GCF_000146045.2_R64_genomic.fna.gz
Connecting to ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)|165.112.9.229|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3843460 (3.7M) [application/x-gzip]
Saving to: ‘GCF_000146045.2_R64_genomic.fna.gz’

     0K .......... .......... .......... .......... ..........  1%  209K 18s
    50K ..........

In [11]:
%ll -hs

total 1.3G
3.7M -rw-rw-r-- 1 gladkov 3.7M Apr  9  2018 GCF_000146045.2_R64_genomic.fna.gz
1.6M -rw-rw-r-- 1 gladkov 1.6M May 16  2020 GCF_000146045.2_R64_genomic.gff.gz
   0 -rw-rw-r-- 1 gladkov    0 Apr  2 02:55 SRR941816.fastq.gz
   0 -rw-rw-r-- 1 gladkov    0 Apr  2 02:56 SRR941816.fastq.gz.1
414M -rw-rw-r-- 1 gladkov 414M Nov 22  2013 SRR941816.fastq.gz.2
456M -rw-rw-r-- 1 gladkov 455M Nov 22  2013 SRR941817.fastq.gz
   0 -rw-rw-r-- 1 gladkov    0 Apr  2 02:57 SRR941818.fastq.gz
 80M -rw-rw-r-- 1 gladkov  80M Nov 22  2013 SRR941818.fastq.gz.1
282M -rw-rw-r-- 1 gladkov 282M Nov 22  2013 SRR941819.fastq.gz


### Выравнивание

In [7]:
%%bash

mv SRR941816.fastq.gz.2 t0_r1.fastq.gz
mv SRR941817.fastq.gz t0_r2.fastq.gz
mv SRR941818.fastq.gz.1 t30_r1.fastq.gz
mv SRR941819.fastq.gz t30_r2.fastq.gz


In [10]:
%ll -ht

total 1.3G
-rw-rw-r-- 1 gladkov    0 Apr  2 05:28 t0_r1.bam
-rw-rw-r-- 1 gladkov    0 Apr  2 05:22 t0_r2.bam
drwxrwxr-x 2 gladkov 4.0K Apr  2 05:22 ref_hisat_ind/
-rw-rw-r-- 1 gladkov    0 Apr  2 02:56 SRR941816.fastq.gz.1
-rw-rw-r-- 1 gladkov 1.6M May 16  2020 GCF_000146045.2_R64_genomic.gff.gz
-rw-rw-r-- 1 gladkov  12M Apr  9  2018 GCF_000146045.2_R64_genomic.fna
-rw-rw-r-- 1 gladkov 3.7M Apr  9  2018 GCF_000146045.2_R64_genomic.fna.gz
-rw-rw-r-- 1 gladkov 282M Nov 22  2013 t30_r2.fastq.gz
-rw-rw-r-- 1 gladkov 455M Nov 22  2013 t0_r2.fastq.gz
-rw-rw-r-- 1 gladkov 414M Nov 22  2013 t0_r1.fastq.gz
-rw-rw-r-- 1 gladkov  80M Nov 22  2013 t30_r1.fastq.gz


In [11]:
%%bash

hisat2 -p 20 -x ref_hisat_ind/ref -U  t0_r1.fastq.gz | samtools sort > t0_r1.bam
hisat2 -p 20 -x ref_hisat_ind/ref -U  t0_r2.fastq.gz | samtools sort > t0_r2.bam
hisat2 -p 20 -x ref_hisat_ind/ref -U  t30_r1.fastq.gz | samtools sort > t30_r1.bam
hisat2 -p 20 -x ref_hisat_ind/ref -U  t30_r2.fastq.gz | samtools sort > t30_r2.bam

9043877 reads; of these:
  9043877 (100.00%) were unpaired; of these:
    512974 (5.67%) aligned 0 times
    7930560 (87.69%) aligned exactly 1 time
    600343 (6.64%) aligned >1 times
94.33% overall alignment rate
[bam_sort_core] merging from 2 files and 1 in-memory blocks...
9929568 reads; of these:
  9929568 (100.00%) were unpaired; of these:
    505209 (5.09%) aligned 0 times
    8645367 (87.07%) aligned exactly 1 time
    778992 (7.85%) aligned >1 times
94.91% overall alignment rate
[bam_sort_core] merging from 2 files and 1 in-memory blocks...
1721675 reads; of these:
  1721675 (100.00%) were unpaired; of these:
    65082 (3.78%) aligned 0 times
    1507992 (87.59%) aligned exactly 1 time
    148601 (8.63%) aligned >1 times
96.22% overall alignment rate
6172452 reads; of these:
  6172452 (100.00%) were unpaired; of these:
    229697 (3.72%) aligned 0 times
    5368109 (86.97%) aligned exactly 1 time
    574646 (9.31%) aligned >1 times
96.28% overall alignment rate
[bam_sort_core]

In [13]:
!ls -htl

total 2.2G
-rw-rw-r-- 1 gladkov gladkov 222M Apr  2 05:37 t30_r2.bam
-rw-rw-r-- 1 gladkov gladkov  67M Apr  2 05:36 t30_r1.bam
-rw-rw-r-- 1 gladkov gladkov 350M Apr  2 05:35 t0_r2.bam
-rw-rw-r-- 1 gladkov gladkov 316M Apr  2 05:32 t0_r1.bam
drwxrwxr-x 2 gladkov gladkov 4.0K Apr  2 05:22 ref_hisat_ind
-rw-rw-r-- 1 gladkov gladkov  770 Apr  2 03:05 draw-heatmap.r
-rw-rw-r-- 1 gladkov gladkov 1.4K Apr  2 03:05 deseq2.r
-rw-rw-r-- 1 gladkov gladkov    0 Apr  2 02:56 SRR941816.fastq.gz.1
-rw-rw-r-- 1 gladkov gladkov 8.9M May 16  2020 GCF_000146045.2_R64_genomic.gff
-rw-rw-r-- 1 gladkov gladkov 1.6M May 16  2020 GCF_000146045.2_R64_genomic.gff.gz
-rw-rw-r-- 1 gladkov gladkov  12M Apr  9  2018 GCF_000146045.2_R64_genomic.fna
-rw-rw-r-- 1 gladkov gladkov 3.7M Apr  9  2018 GCF_000146045.2_R64_genomic.fna.gz
-rw-rw-r-- 1 gladkov gladkov 282M Nov 22  2013 t30_r2.fastq.gz
-rw-rw-r-- 1 gladkov gladkov 455M Nov 22  2013 t0_r2.fastq.gz
-rw-rw-r-- 1 gladkov gladkov 414M Nov 22  2013 t0_r1.fastq.gz
-rw

In [16]:
%%bash

# gffread GCF_000146045.2_R64_genomic.gff -T -o ref.gtf
featureCounts -g gene_id -a ref.gtf -o ref_ann_count.txt *bam
cat ref_ann_count.txt | cut -f 1,7-10 > simple_counts.txt




        ==========     _____ _    _ ____  _____  ______          _____  
        =====         / ____| |  | |  _ \|  __ \|  ____|   /\   |  __ \ 
          =====      | (___ | |  | | |_) | |__) | |__     /  \  | |  | |
            ====      \___ \| |  | |  _ <|  _  /|  __|   / /\ \ | |  | |
              ====    ____) | |__| | |_) | | \ \| |____ / ____ \| |__| |
        ==========   |_____/ \____/|____/|_|  \_\______/_/    \_\_____/
	  v2.0.1

//========================== featureCounts setting ===========================\\
||                                                                            ||
||             Input files : 4 BAM files                                      ||
||                           o t0_r1.bam                                      ||
||                           o t0_r2.bam                                      ||
||                           o t30_r1.bam                                     ||
||                           o t30_r2.bam                        

In [17]:
!ls -htl

total 2.2G
-rw-rw-r-- 1 gladkov gladkov 174K Apr  2 05:58 simple_counts.txt
-rw-rw-r-- 1 gladkov gladkov 387K Apr  2 05:58 ref_ann_count.txt
-rw-rw-r-- 1 gladkov gladkov  563 Apr  2 05:58 ref_ann_count.txt.summary
-rw-rw-r-- 1 gladkov gladkov   59 Apr  2 05:56 temp-core-024926-002590FCB132.sam
-rw-rw-r-- 1 gladkov gladkov 2.3M Apr  2 05:45 ref.gtf
-rw-rw-r-- 1 gladkov gladkov 222M Apr  2 05:37 t30_r2.bam
-rw-rw-r-- 1 gladkov gladkov  67M Apr  2 05:36 t30_r1.bam
-rw-rw-r-- 1 gladkov gladkov 350M Apr  2 05:35 t0_r2.bam
-rw-rw-r-- 1 gladkov gladkov 316M Apr  2 05:32 t0_r1.bam
drwxrwxr-x 2 gladkov gladkov 4.0K Apr  2 05:22 ref_hisat_ind
-rw-rw-r-- 1 gladkov gladkov  770 Apr  2 03:05 draw-heatmap.r
-rw-rw-r-- 1 gladkov gladkov 1.4K Apr  2 03:05 deseq2.r
-rw-rw-r-- 1 gladkov gladkov    0 Apr  2 02:56 SRR941816.fastq.gz.1
-rw-rw-r-- 1 gladkov gladkov 8.9M May 16  2020 GCF_000146045.2_R64_genomic.gff
-rw-rw-r-- 1 gladkov gladkov 1.6M May 16  2020 GCF_000146045.2_R64_genomic.gff.gz
-rw-rw-r-- 1

### DESeq

Использование кастомного скрипта для диффэкспрессии и визуализации

In [ ]:
%%bash

cat simple_counts.txt | R -f deseq2.r
cat norm-matrix-deseq2.txt | R -f draw-heatmap.r

In [21]:
!ls -htl

total 2.2G
-rw-rw-r-- 1 gladkov gladkov 276K Apr  2 06:02 output.pdf
-rw-rw-r-- 1 gladkov gladkov 476K Apr  2 06:01 norm-matrix-deseq2.txt
-rw-rw-r-- 1 gladkov gladkov 814K Apr  2 06:01 result.txt
-rw-rw-r-- 1 gladkov gladkov 174K Apr  2 05:58 simple_counts.txt
-rw-rw-r-- 1 gladkov gladkov 387K Apr  2 05:58 ref_ann_count.txt
-rw-rw-r-- 1 gladkov gladkov  563 Apr  2 05:58 ref_ann_count.txt.summary
-rw-rw-r-- 1 gladkov gladkov   59 Apr  2 05:56 temp-core-024926-002590FCB132.sam
-rw-rw-r-- 1 gladkov gladkov 2.3M Apr  2 05:45 ref.gtf
-rw-rw-r-- 1 gladkov gladkov 222M Apr  2 05:37 t30_r2.bam
-rw-rw-r-- 1 gladkov gladkov  67M Apr  2 05:36 t30_r1.bam
-rw-rw-r-- 1 gladkov gladkov 350M Apr  2 05:35 t0_r2.bam
-rw-rw-r-- 1 gladkov gladkov 316M Apr  2 05:32 t0_r1.bam
drwxrwxr-x 2 gladkov gladkov 4.0K Apr  2 05:22 ref_hisat_ind
-rw-rw-r-- 1 gladkov gladkov  770 Apr  2 03:05 draw-heatmap.r
-rw-rw-r-- 1 gladkov gladkov 1.4K Apr  2 03:05 deseq2.r
-rw-rw-r-- 1 gladkov gladkov    0 Apr  2 02:56 SRR94181

Экспорт первых 50 последовательностей для GO

In [43]:
!head result.txt -n 50 | cut -f 1 > genes.txt

In [44]:
df = pd.read_table("genes.txt", header=None)
df[0] = df[0].str[5:]
df.to_csv("genes.txt", sep='\t', index=False, header=False)

In [80]:
df = pd.read_table("result.txt")
df['id'] = df['Unnamed: 0'].str[5:]
df_dec = df.query("padj < 0.05" and "log2FoldChange < 0")
df_dec

,Unnamed: 0,id,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
27,gene-YLR327C,YLR327C,14481.959870,-5.025613,0.391275,-12.844196,9.270986e-38,2.002533e-35
46,gene-YKR097W,YKR097W,21637.671931,-4.708130,0.391536,-12.024764,2.633401e-33,3.388683e-31
67,gene-YMR081C,YMR081C,2812.519586,-4.794055,0.428029,-11.200311,4.063053e-29,3.613727e-27
78,gene-YDR342C,YDR342C,2155.471247,-4.507558,0.408420,-11.036585,2.545237e-28,1.948556e-26
95,gene-YCR021C,YCR021C,60798.803813,-4.451403,0.418098,-10.646801,1.804612e-26,1.145327e-24
...,...,...,...,...,...,...,...,...
6036,gene-YDR118W,YDR118W,45.984862,-0.009579,0.622037,-0.015399,9.877138e-01,9.922505e-01
6040,gene-YHR042W,YHR042W,643.267540,-0.004110,0.420144,-0.009783,9.921942e-01,9.933439e-01
6041,gene-YBR156C,YBR156C,83.502575,-0.004319,0.560919,-0.007701,9.938559e-01,9.948428e-01
6044,gene-YHR187W,YHR187W,304.190525,-0.002411,0.424926,-0.005675,9.954723e-01,9.959663e-01


In [95]:
df_inc = df.query("padj < 0.05" and "log2FoldChange > 0").sort_values('padj', ascending=[1])
df_inc

,Unnamed: 0,id,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
0,gene-YER062C,YER062C,16729.173256,7.899389,0.407321,19.393515,8.754485e-84,5.294713e-80
1,gene-YDR536W,YDR536W,10087.918955,7.873736,0.410494,19.181142,5.319930e-82,1.608747e-78
2,gene-YHR094C,YHR094C,8495.018858,7.881149,0.431516,18.263876,1.604837e-74,3.235352e-71
3,gene-YNL065W,YNL065W,2109.430343,7.784779,0.466558,16.685559,1.669372e-62,2.524091e-59
4,gene-YKL120W,YKL120W,1727.028368,7.268735,0.482388,15.068223,2.620681e-51,3.169976e-48
...,...,...,...,...,...,...,...,...
6039,gene-YPR176C,YPR176C,254.777461,0.004336,0.433361,0.010006,9.920169e-01,9.933308e-01
6042,gene-YHR140W,YHR140W,52.211445,0.004606,0.622473,0.007400,9.940955e-01,9.949180e-01
6043,gene-YDL188C,YDL188C,199.186378,0.003418,0.491618,0.006952,9.944531e-01,9.951112e-01
6045,gene-YML038C,YML038C,184.081372,0.002300,0.492947,0.004667,9.962766e-01,9.966061e-01


Соответствие результатов DESeq и GO
Несбалансированно - далее дробим датасет и отправляем в http://www.yeastgenome.org/cgi-bin/GO/goSlimMapper.pl.

In [94]:
list_to_grep = df_inc["id"]
def _grep(x, list_to_grep):
    """takes a string (x) and checks whether any string from a given 
       list of strings (list_to_grep) exists in `x`"""
    for text in list_to_grep:
        if text.lower() in x.lower():
            return True
    return False
mask = mapp.ANNOTATED_GENES.apply(_grep, list_to_grep=list_to_grep)
mapp[mask]

,GOID,TERM,NUM_LIST_ANNOTATIONS,LIST_SIZE,CLUSTER_FREQUENCY,TOTAL_NUM_ANNOTATIONS,POPULATION_SIZE,GENOME_FREQUENCY,ANNOTATED_GENES
0,GO:0006364,rRNA processing,13,49,26.53%,366,6443,5.68%,"YDR449C, YEL026W, YER127W, YGR159C, YHR066W, Y..."
1,GO:0042273,ribosomal large subunit biogenesis,9,49,18.37%,130,6443,2.02%,"YCR072C, YDL063C, YEL026W, YHR066W, YIR012W, Y..."
2,GO:0042274,ribosomal small subunit biogenesis,8,49,16.33%,146,6443,2.27%,"YDR449C, YEL026W, YER127W, YGR159C, YHR196W, Y..."
3,GO:0042255,ribosome assembly,7,49,14.29%,79,6443,1.23%,"YCR072C, YGR159C, YHR066W, YIR012W, YLR264W, Y..."
4,GO:0006360,transcription by RNA polymerase I,6,49,12.24%,71,6443,1.10%,"YHR196W, YJL148W, YJR063W, YML043C, YMR093W, Y..."
5,GO:0006811,ion transport,6,49,12.24%,340,6443,5.28%,"YDR536W, YHR094C, YKL120W, YNL065W, YNR060W, Y..."
6,GO:0055085,transmembrane transport,5,49,10.20%,468,6443,7.26%,"YDR536W, YHR094C, YKL120W, YNL065W, YOR271C"
7,GO:0055086,nucleobase-containing small molecule metabolic...,4,49,8.16%,220,6443,3.41%,"YBL039C, YMR300C, YNL141W, YOL136C"
8,GO:0005975,carbohydrate metabolic process,4,49,8.16%,253,6443,3.93%,"YBR105C, YER062C, YKR097W, YOL136C"
9,GO:0006417,regulation of translation,3,49,6.12%,234,6443,3.63%,"YLR264W, YNL112W, YOR359W"


In [93]:
list_to_grep_2 = df_dec["id"]
mask_2 = mapp.ANNOTATED_GENES.apply(_grep, list_to_grep=list_to_grep_2)
mapp[mask_2]

,GOID,TERM,NUM_LIST_ANNOTATIONS,LIST_SIZE,CLUSTER_FREQUENCY,TOTAL_NUM_ANNOTATIONS,POPULATION_SIZE,GENOME_FREQUENCY,ANNOTATED_GENES
8,GO:0005975,carbohydrate metabolic process,4,49,8.16%,253,6443,3.93%,"YBR105C, YER062C, YKR097W, YOL136C"


In [71]:
mapp = pd.read_table("mapper_genes_26968_slimTab.txt")
mapp

,GOID,TERM,NUM_LIST_ANNOTATIONS,LIST_SIZE,CLUSTER_FREQUENCY,TOTAL_NUM_ANNOTATIONS,POPULATION_SIZE,GENOME_FREQUENCY,ANNOTATED_GENES
0,GO:0006364,rRNA processing,13,49,26.53%,366,6443,5.68%,"YDR449C, YEL026W, YER127W, YGR159C, YHR066W, Y..."
1,GO:0042273,ribosomal large subunit biogenesis,9,49,18.37%,130,6443,2.02%,"YCR072C, YDL063C, YEL026W, YHR066W, YIR012W, Y..."
2,GO:0042274,ribosomal small subunit biogenesis,8,49,16.33%,146,6443,2.27%,"YDR449C, YEL026W, YER127W, YGR159C, YHR196W, Y..."
3,GO:0042255,ribosome assembly,7,49,14.29%,79,6443,1.23%,"YCR072C, YGR159C, YHR066W, YIR012W, YLR264W, Y..."
4,GO:0006360,transcription by RNA polymerase I,6,49,12.24%,71,6443,1.10%,"YHR196W, YJL148W, YJR063W, YML043C, YMR093W, Y..."
5,GO:0006811,ion transport,6,49,12.24%,340,6443,5.28%,"YDR536W, YHR094C, YKL120W, YNL065W, YNR060W, Y..."
6,GO:0055085,transmembrane transport,5,49,10.20%,468,6443,7.26%,"YDR536W, YHR094C, YKL120W, YNL065W, YOR271C"
7,GO:0055086,nucleobase-containing small molecule metabolic...,4,49,8.16%,220,6443,3.41%,"YBL039C, YMR300C, YNL141W, YOL136C"
8,GO:0005975,carbohydrate metabolic process,4,49,8.16%,253,6443,3.93%,"YBR105C, YER062C, YKR097W, YOL136C"
9,GO:0006417,regulation of translation,3,49,6.12%,234,6443,3.63%,"YLR264W, YNL112W, YOR359W"


### Дробим

Импорт/экспорт через pandas

In [98]:
df_inc.head(50).id.to_csv("res_inc.txt", sep='\t', index=False, header=False)
df_dec.head(50).id.to_csv("res_dec.txt", sep='\t', index=False, header=False)

In [100]:
!wget http://gotermfinder.yeastgenome.org/mapper_genes_2615_slimTab.txt -O go_inc.txt


--2021-04-02 21:22:34--  http://gotermfinder.yeastgenome.org/mapper_genes_2615_slimTab.txt
Resolving gotermfinder.yeastgenome.org (gotermfinder.yeastgenome.org)... 35.161.238.106
Connecting to gotermfinder.yeastgenome.org (gotermfinder.yeastgenome.org)|35.161.238.106|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3848 (3.8K) [text/plain]
Saving to: ‘go_inc.txt’

go_inc.txt          100%[===================>]   3.76K  --.-KB/s    in 0s      

2021-04-02 21:22:35 (281 MB/s) - ‘go_inc.txt’ saved [3848/3848]



In [101]:
!wget http://gotermfinder.yeastgenome.org/mapper_genes_2634_slimTab.txt -O go_dec.txt

--2021-04-02 21:23:56--  http://gotermfinder.yeastgenome.org/mapper_genes_2634_slimTab.txt
Resolving gotermfinder.yeastgenome.org (gotermfinder.yeastgenome.org)... 35.161.238.106
Connecting to gotermfinder.yeastgenome.org (gotermfinder.yeastgenome.org)|35.161.238.106|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4449 (4.3K) [text/plain]
Saving to: ‘go_dec.txt’

go_dec.txt          100%[===================>]   4.34K  --.-KB/s    in 0s      

2021-04-02 21:23:57 (394 MB/s) - ‘go_dec.txt’ saved [4449/4449]



In [107]:
go_inc = pd.read_table("go_inc.txt")
go_inc.sort_values("NUM_LIST_ANNOTATIONS", ascending=False)

,GOID,TERM,NUM_LIST_ANNOTATIONS,LIST_SIZE,CLUSTER_FREQUENCY,TOTAL_NUM_ANNOTATIONS,POPULATION_SIZE,GENOME_FREQUENCY,ANNOTATED_GENES
0,GO:0006364,rRNA processing,15,50,30.00%,366,6443,5.68%,"YDR449C, YEL026W, YER127W, YGR128C, YGR159C, Y..."
1,GO:0042274,ribosomal small subunit biogenesis,10,50,20.00%,146,6443,2.27%,"YDR449C, YEL026W, YER127W, YGR128C, YGR159C, Y..."
2,GO:0042273,ribosomal large subunit biogenesis,9,50,18.00%,130,6443,2.02%,"YCR072C, YDL063C, YEL026W, YHR066W, YIR012W, Y..."
3,GO:0006360,transcription by RNA polymerase I,8,50,16.00%,71,6443,1.10%,"YGR128C, YHR196W, YJL148W, YJR063W, YKL078W, Y..."
4,GO:0042255,ribosome assembly,7,50,14.00%,79,6443,1.23%,"YCR072C, YGR159C, YHR066W, YIR012W, YLR264W, Y..."
5,GO:0006811,ion transport,6,50,12.00%,340,6443,5.28%,"YDR536W, YHR094C, YKL120W, YNL065W, YNR060W, Y..."
6,GO:0055085,transmembrane transport,5,50,10.00%,468,6443,7.26%,"YDR536W, YHR094C, YKL120W, YNL065W, YOR271C"
7,GO:0006401,RNA catabolic process,4,50,8.00%,166,6443,2.58%,"YER049W, YLR264W, YNL112W, YOR359W"
8,GO:0006417,regulation of translation,4,50,8.00%,234,6443,3.63%,"YER049W, YLR264W, YNL112W, YOR359W"
9,GO:0055086,nucleobase-containing small molecule metabolic...,4,50,8.00%,220,6443,3.41%,"YBL039C, YMR300C, YNL141W, YOL136C"


In [108]:
go_dec = pd.read_table("go_dec.txt")
go_dec.sort_values("NUM_LIST_ANNOTATIONS", ascending=False)

,GOID,TERM,NUM_LIST_ANNOTATIONS,LIST_SIZE,CLUSTER_FREQUENCY,TOTAL_NUM_ANNOTATIONS,POPULATION_SIZE,GENOME_FREQUENCY,ANNOTATED_GENES
0,GO:0005975,carbohydrate metabolic process,13,50,26.00%,253,6443,3.93%,"YBR053C, YBR149W, YCL040W, YEL011W, YFR015C, Y..."
2,GO:0045333,cellular respiration,7,50,14.00%,116,6443,1.80%,"YKL085W, YKL109W, YLR174W, YLR304C, YMR081C, Y..."
3,GO:0042221,response to chemical,7,50,14.00%,530,6443,8.23%,"YCR021C, YDR171W, YDR216W, YGR008C, YKL109W, Y..."
4,GO:0006629,lipid metabolic process,7,50,14.00%,348,6443,5.40%,"YGR157W, YJR073C, YKL182W, YLR133W, YNR016C, Y..."
5,GO:0006457,protein folding,7,50,14.00%,121,6443,1.88%,"YBR072W, YBR169C, YDR171W, YMR186W, YNL077W, Y..."
1,GO:0032787,monocarboxylic acid metabolic process,7,50,14.00%,164,6443,2.55%,"YCL040W, YKL182W, YLR174W, YNL117W, YNR016C, Y..."
6,GO:0009408,response to heat,6,50,12.00%,67,6443,1.04%,"YBR072W, YCR021C, YDR171W, YMR186W, YPL004C, Y..."
7,GO:0055085,transmembrane transport,6,50,12.00%,468,6443,7.26%,"YCL040W, YCR021C, YDR342C, YDR343C, YLR411W, Y..."
8,GO:0006811,ion transport,5,50,10.00%,340,6443,5.28%,"YCR021C, YDR342C, YDR343C, YLR411W, YNL055C"
11,GO:0007005,mitochondrion organization,4,50,8.00%,287,6443,4.45%,"YLR304C, YNL055C, YOR027W, YPL240C"
